In [1]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

import pandas as pd

%time


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [2]:
train_len = 40000
start_time = 0
test_length = 20000
nb_generations = 10000

seed_timesteps = test_length 


In [3]:
import os
import pandas as pd

# opt_shift_hour の異なる値
opt_shift_hours = [-7, 0, 7, 'random']

# 各opt_shift_hourのためのnrmse値を格納する辞書
nrmse_values_per_dimension = {}

for opt_shift_hour in opt_shift_hours:
    # ディレクトリの作成
    dir_name_plot = f"nrmse_plot/plot_{opt_shift_hour}"
    os.makedirs(dir_name_plot, exist_ok=True)

    # 特定のopt_shift_hourのためのnrmse値リストの初期化
    nrmse_values_per_dimension[opt_shift_hour] = []

    for shift_hour in range(-12, 13):
        # データの読み込み
        filename_with_force_result = f"opt_{opt_shift_hour}_VDP_gen/VDP_{shift_hour}.csv"
        data_loaded_with_force_result = pd.read_csv(filename_with_force_result, header=None)
        X_result = data_loaded_with_force_result.iloc[:, [0, 1, 2]].values

        filename_with_force_val = f"opt_{opt_shift_hour}_VDP_val/VDP_{shift_hour}.csv"
        data_loaded_with_force_val = pd.read_csv(filename_with_force_val, header=None)
        X_val = data_loaded_with_force_val.iloc[:, [0, 1, 2]].values

        X_t = X_val[:nb_generations]
        X_gen = X_result[:nb_generations]

        num_dimensions = X_gen.shape[1]

        nrmse_values = []  # 現在のshift_hourのためのnrmse値

        for dim in range(num_dimensions):
            # 各次元に対するnrmseの計算
            nrmse_val = nrmse(X_gen[:, dim][:nb_generations], X_t[:, dim][:nb_generations])
            nrmse_values.append(nrmse_val)

        # 各次元のnrmse値を保存
        nrmse_values_per_dimension[opt_shift_hour].append(nrmse_values)

# nrmse_values_per_dimensionには各opt_shift_hourごとに各shift_hourの3つのnrmse値が含まれています
nrmse_values_per_dimension


{-7: [[0.21096239075637105, 0.4189570562943388, 0.6055222543306455],
  [0.3045819345820202, 0.20471531161437498, 0.5359422229820356],
  [0.26424780111015483, 0.2848539039227779, 0.48267750777017204],
  [0.19082608150539862, 0.20799079852560703, 0.3053755190866705],
  [0.018511768699101042, 0.017896936533095122, 0.10331645456642281],
  [0.03439841147421159, 0.03345661318159286, 0.06852964153184628],
  [0.037178010689031066, 0.036239255984347886, 0.12505993145568767],
  [0.03587799608804343, 0.03613499209268104, 0.173534138304794],
  [0.03774379147057668, 0.038280780394634006, 0.24251996051065378],
  [0.045227432732630045, 0.04543036528045772, 0.28548062029927795],
  [0.05295923465269389, 0.05324938521975386, 0.2970623036438329],
  [0.059172991372563113, 0.05869191941509314, 0.29654218446235103],
  [0.051788750260583015, 0.05322139860986483, 0.3087947720956594],
  [0.24375768598085018, 0.3180105892677463, 0.43638236754248405],
  [0.2537765049700102, 0.32907000804920333, 0.470256834638183

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


# opt_shift_hours の異なる値
opt_shift_hours = [-7, 0, 7, 'random']

# shift_hour の範囲
shift_hours = range(-12, 13)

# 図の作成
plt.figure(figsize=(18, 6))

# 各次元ごとにサブプロットを作成
for dim in range(3):  # 3次元のデータを仮定
    ax = plt.subplot(1, 3, dim + 1)
    
    for opt_shift_hour in opt_shift_hours:
        # 各opt_shift_hourごとにnrmse値を取得してプロット
        nrmse_values = [nrmse_value[dim] for nrmse_value in nrmse_values_per_dimension[opt_shift_hour]]
        ax.plot(shift_hours, nrmse_values, label=f'opt_shift_hour = {opt_shift_hour}')
    
    ax.set_title(f'Dimension {dim + 1}')
    ax.set_xlabel('Shift Hour')
    ax.set_ylabel('NRMSE')
    ax.grid(True)

# 凡例とグラフの表示
plt.legend()
plt.tight_layout()

# PDF形式で保存
pdf_file_path = 'nrmse_plot/plot.pdf'  # 保存先のパスを設定
with PdfPages(pdf_file_path) as pdf:
    pdf.savefig()  # 現在の図をPDFに保存
    plt.close()  # 図を閉じる


In [11]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# opt_shift_hours の異なる値
opt_shift_hours = [-7, 0, 7, 'random']

# shift_hour の範囲
shift_hours = range(-12, 13)

# PDFファイルの作成
pdf_filename = "nrmse_plots.pdf"
with PdfPages(pdf_filename) as pdf:
    # 図の作成
    fig, axes = plt.subplots(3, 1, figsize=(18, 12))  # サイズを調整

    titles = ['X', 'Y', 'P']  # タイトルをリストで指定
    lines = []  # 凡例用のラインを格納するリスト
    for dim, ax in enumerate(axes):
        for opt_shift_hour in opt_shift_hours:
            nrmse_values = [nrmse_value[dim] for nrmse_value in nrmse_values_per_dimension[opt_shift_hour]]
            line, = ax.plot(shift_hours, nrmse_values, label=f'opt_shift_hour = {opt_shift_hour}')
            if dim == 0:  # 一番最初のサブプロットでラインを保存
                lines.append(line)

        ax.set_title(titles[dim], fontsize = 18)  # タイトルを設定
        ax.set_xlabel('Shift hour', fontsize = 18)
        ax.set_ylabel('NRMSE', fontsize = 18)
        ax.grid(True)


    # 共通の凡例を図の下に配置
    fig.legend(lines, [f'opt_shift_hour = {opt_shift_hour}' for opt_shift_hour in opt_shift_hours], 
               loc='lower center', bbox_to_anchor=(0.5, 0.1), ncol=len(opt_shift_hours), fontsize=18)

    # グラフが凡例に重ならないように余白を調整
    plt.tight_layout(rect=[0, 0.2, 1, 1])  # 下の余白を大きくする

# PDF形式で保存
pdf_file_path = 'nrmse_plot/nrmse_plot.pdf'  # 保存先のパスを設定
with PdfPages(pdf_file_path) as pdf:
    pdf.savefig()  # 現在の図をPDFに保存
    plt.close()  # 図を閉じる
